In [1]:
import numpy as np 
import pandas as pd 
import os

## Importing Libraries

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd
from tensorflow.keras.models import load_model

print( tf.version)


<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.8/dist-packages/tensorflow/_api/v2/version/__init__.py'>


## Read the data

In [3]:
data_path='chatbot-training-dataset'

In [4]:
#A list is created combining file paths and folder paths.
import os
data_paths = [os.path.join(pth, f) 
for pth, dirs, files in os.walk(data_path) for f in files]
data_paths

[]

In [5]:
data_path1='chatbot dataset.txt'

In [6]:
#reads the file content and splits it into columns.
#The input text and target text variables are assigned
input_texts = []
target_texts = []
with open(data_path1) as f:
    lines = f.read().split('\n')
for line in lines[: min(600, len(lines) - 1)]:
    input_text = line.split('\t')[0]
    target_text = line.split('\t')[1]
    input_texts.append(input_text)
    target_texts.append(target_text)

In [7]:
print('type of input_text',type(input_text))
print('type of target_texts',type(target_texts))

type of input_text <class 'str'>
type of target_texts <class 'list'>


In [8]:
##converting the list in pandas dataframe since input_text,target_text are both are type of list
zippedList =  list(zip(input_texts, target_texts))
lines = pd.DataFrame(zippedList, columns = ['input' , 'output']) 
lines.head()

,input,output
0,What are your interests,I am interested in all kinds of things. We can...
1,What are your favorite subjects,"My favorite subjects include robotics, compute..."
2,What are your interests,"I am interested in a wide variety of topics, a..."
3,What is your number,I don't have any number
4,What is your number,23 skiddoo!


## Preparing input data for the Encoder

In [9]:
#create a list and insert the lines.tokenize the input text.
#then create a new list and string lenghts are entered.
#find the max input lenght and makes all input same size

input_lines = list()
for line in lines.input:
    input_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( input_lines ) 
tokenized_input_lines = tokenizer.texts_to_sequences( input_lines ) 

length_list = list()
for token_seq in tokenized_input_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'Input max length is {}'.format( max_input_length ))

padded_input_lines = preprocessing.sequence.pad_sequences( tokenized_input_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_input_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

input_word_dict = tokenizer.word_index
num_input_tokens = len( input_word_dict )+1
print( 'Number of Input tokens = {}'.format( num_input_tokens))

Input max length is 22
Encoder input data shape -> (566, 22)
Number of Input tokens = 518


In [10]:
encoder_input_data

array([[  4,   3,  12, ...,   0,   0,   0],
       [  4,   3,  12, ...,   0,   0,   0],
       [  4,   3,  12, ...,   0,   0,   0],
       ...,
       [498, 499,   9, ...,   0,   0,   0],
       [  9, 207, 208, ...,   0,   0,   0],
       [510, 192,   5, ...,   0,   0,   0]], dtype=int32)

## Preparing output data for the Decoder


In [11]:
#create a list and adds squences, converts to token strings
#then create a new list and string lenghts are entered.
#find the max input lenght , pads the tokenized_output_lines to max_output_length
#converted to a numpy array and stored in decoder_input_data.
output_lines = list()
for line in lines.output:
    output_lines.append( '<START> ' + line + ' <END>' )  

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( output_lines ) 
tokenized_output_lines = tokenizer.texts_to_sequences( output_lines ) 

length_list = list()
for token_seq in tokenized_output_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'Output max length is {}'.format( max_output_length ))

padded_output_lines = preprocessing.sequence.pad_sequences( tokenized_output_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_output_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

output_word_dict = tokenizer.word_index
num_output_tokens = len( output_word_dict )+1
print( 'Number of Output tokens = {}'.format( num_output_tokens))


Output max length is 74
Decoder input data shape -> (566, 74)
Number of Output tokens = 1692


## Preparing target data for the Decoder 

In [12]:
# create a list and add the tokens without 1
# turns into one-hot encoded 
decoder_target_data = list()
for token_seq in tokenized_output_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_output_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_output_lines = utils.to_categorical( padded_output_lines , num_output_tokens )
decoder_target_data = np.array(onehot_output_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (566, 74, 1692)


## Defining the Model


In [13]:
#We use 2 inputs which are encoder and decoder inputs.
#The model is then compiled with the Adam optimizer and the categorical_crossentropy loss function.
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_input_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_output_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_output_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')
# is used to print out a summary of the model's architecture, including the number of parameters in each layer and the overall number of parameters in the model.
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    132608      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    433152      ['input_2[0][0]']                
                                                                                              

## Training

This is a built-in function provided by the Keras library, which can be used to save the trained model and load it later without the need to retrain it again.For example, many machine learning models are saved and loaded in h5 format.

In [14]:
#This is a built-in function provided by the Keras library, which can be used to save the trained model and 
#load it later without the need to retrain it again.For example, many machine learning models are saved and loaded in h5 format.
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=124, epochs=500) 
model.save( 'model.h5' ) 
#model=load_model('model.h5')

Epoch 1/500
5/5 [==============================] - 11s 516ms/step - loss: 1.4406
Epoch 2/500
5/5 [==============================] - 2s 490ms/step - loss: 1.4311
Epoch 3/500
5/5 [==============================] - 2s 492ms/step - loss: 1.3439
Epoch 4/500
5/5 [==============================] - 3s 511ms/step - loss: 1.1874
Epoch 5/500
5/5 [==============================] - 3s 517ms/step - loss: 1.1432
Epoch 6/500
5/5 [==============================] - 2s 496ms/step - loss: 1.1232
Epoch 7/500
5/5 [==============================] - 2s 492ms/step - loss: 1.1103
Epoch 8/500
5/5 [==============================] - 3s 515ms/step - loss: 1.1049
Epoch 9/500
5/5 [==============================] - 2s 496ms/step - loss: 1.0987
Epoch 10/500
5/5 [==============================] - 3s 507ms/step - loss: 1.0930
Epoch 11/500
5/5 [==============================] - 3s 500ms/step - loss: 1.0873
Epoch 12/500
5/5 [==============================] - 2s 499ms/step - loss: 1.0802
Epoch 13/500
5/5 [==================

## Inference models

In [15]:
#Creating a function for encoder and decoder models
#There are layers and these layers represent the initial states of the decoder LSTM layer.
#This function returns the encoder_model and decoder_model variables
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [16]:
#Function used to convert a given sentence into a sequence of tokens.
#This function returns the formatted token sequence.
import tensorflow as tf
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

In [ ]:
#Asking questions and model will start the answers using our dataset.
enc_model , dec_model = make_inference_models()
for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = output_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in output_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Bot:" +decoded_translation.replace(' end', '') )
    print()

User: What is your favorite number
1/1 [==============================] - 0s 18ms/step
Bot: i find i'm quite fond of the number 42

User: Why can't you eat food
1/1 [==============================] - 0s 16ms/step
Bot: i'm a software program i blame the hardware

